let’s do a **practical, copy-pasteable, step-by-step guide** for both approaches Snowflake supports for *automatic* external-table refresh on AWS S3:

* **Option A — SQS (simple and common)**: Snowflake provides an SQS queue ARN for your external table; you wire your S3 bucket event notifications to that queue. Snowflake listens to that queue and automatically refreshes the external table metadata. ([Snowflake Documentation][1])

* **Option B — SNS (broadcaster / existing-notifications)**: Use SNS as a broadcaster so multiple subscribers (Lambda, other SQS, Snowflake) can get the same S3 events. You’ll create an SNS topic, add the Snowflake-generated policy to the topic access policy (Snowflake helps you generate it), then create the external table pointing to that SNS topic (or subscribe Snowflake’s SQS queue to it). ([Snowflake Documentation][2])

Below I walk you through both options (AWS console steps + Snowflake SQL), verification, and common gotchas.

---

# Quick prerequisites & rules of thumb

* Your Snowflake account must be on **AWS** to use these features. ([Snowflake Documentation][1])
* You must have **admin access** to the target AWS account (create SNS/SQS, change bucket notifications, edit SNS policy). ([Snowflake Documentation][1])
* Best practice: create the S3 event filter (prefix/suffix) narrow enough to avoid noise/cost. AWS forbids overlapping queue configs for the same prefix. ([Snowflake Documentation][1])

---

# Option A — Use Snowflake’s SQS queue (recommended when you don’t already have SNS)

This is the easiest: Snowflake provides the SQS queue ARN, you configure the S3 bucket to push events to that SQS queue, and Snowflake will auto-refresh your external table metadata.

**High level flow**

1. Create stage & external table in Snowflake (AUTO\_REFRESH is `TRUE` by default).
2. Run `SHOW EXTERNAL TABLES;` and copy the `notification_channel` SQS ARN.
3. In AWS S3 console, create an event notification that sends `ObjectCreated` (and `ObjectRemoved`) events to that SQS ARN.
4. Run one `ALTER EXTERNAL TABLE ... REFRESH;` to seed metadata — after that notifications trigger automatic refreshes. ([Snowflake Documentation][1])

**Step-by-step (with examples)**

1. Create stage (example):

```sql
USE SCHEMA mydb.public;

CREATE STAGE mystage
  URL = 's3://mybucket/path'
  STORAGE_INTEGRATION = my_storage_int;
```

2. Create external table:

```sql
CREATE OR REPLACE EXTERNAL TABLE ext_table
  WITH LOCATION = @mystage/somepath/
  FILE_FORMAT = (TYPE = PARQUET);
-- AUTO_REFRESH = TRUE is default (unless PARTITION_TYPE=USER_SPECIFIED)
```

(If you have partitioning expressions, define them here; note manual partitions disable auto refresh.)

3. Get the SQS queue ARN that Snowflake wants you to wire S3 to:

```sql
SHOW EXTERNAL TABLES;
-- look at the notification_channel column — copy that ARN (it’s an SQS ARN)
```

`notification_channel` = the ARN of Snowflake’s SQS queue for your external-table region/account. ([Snowflake Documentation][3])

4. In AWS (S3 bucket):

* Open the **S3 console → Buckets → your-bucket → Properties → Event notifications**.
* **Create event notification**:

  * Name: `AutoRefresh-Snowflake` (or anything)
  * Events: **ObjectCreated (All)** and **ObjectRemoved** (or choose specific ones)
  * Destination: **SQS queue**
  * SQS queue ARN: **paste the notification\_channel ARN** you copied from `SHOW EXTERNAL TABLES`
  * Optionally add prefix/suffix filters (e.g., `path/2025/09/`) to narrow events.
    Save. (AWS doc on S3 event notifications). ([AWS Documentation][4])

5. Manually seed metadata once:

```sql
ALTER EXTERNAL TABLE ext_table REFRESH;
```

After this one manual refresh the S3 → SQS events will automatically trigger future refreshes. ([Snowflake Documentation][1])

**Verify**

* `SHOW EXTERNAL TABLES;` — `last_refreshed_on` should update after an S3 PUT/DELETE. ([Snowflake Documentation][3])
* Check SQS queue in AWS Console (Messages in queue) — should see S3 event messages if an object was created.
* If table is not updating, run `ALTER SESSION SET USE_CACHED_RESULT = FALSE;` and re-query (to avoid result cache confusion).

**Common gotchas**

* Snowflake designates *no more than one* Snowflake SQS queue per AWS region; the same queue can service multiple buckets/prefixes. Don’t create conflicting S3 notifications (AWS forbids overlapping queue configs). ([Snowflake Documentation][1])
* If your external table uses **user-specified partitions** (`PARTITION_TYPE = USER_SPECIFIED`), `AUTO_REFRESH = TRUE` is **not supported** — you must manage partitions manually. (See CREATE EXTERNAL TABLE docs.) ([Snowflake Documentation][5])

---

# Option B — Use SNS as a broadcaster (if you already use SNS or share notifications)

Use this when S3 already publishes to an SNS topic or when multiple subscribers need the same events.

**High level flow**

1. Create SNS topic (or use existing topic) and note ARN.
2. Use Snowflake to generate the little IAM policy snippet that *lets Snowflake’s SQS queue subscribe to your SNS topic* (call `SYSTEM$GET_AWS_SNS_IAM_POLICY`). ([Snowflake Documentation][6])
3. Merge that policy into the SNS topic access policy in the AWS console (so Snowflake’s SQS can subscribe).
4. Create external table with `AWS_SNS_TOPIC = '<your_sns_arn>'` (this causes Snowflake to subscribe its SQS queue to your SNS topic).
5. Run `ALTER EXTERNAL TABLE ... REFRESH;` once to seed metadata. ([Snowflake Documentation][1])

**Step-by-step (with examples)**

1. **Create SNS topic** (AWS Console → SNS → Topics → Create topic) — use **Standard** topic (not FIFO). Note the Topic ARN (e.g. `arn:aws:sns:us-east-1:111122223333:mytopic`). (AWS docs). ([AWS Documentation][4])

2. **(Optional) Subscribe other endpoints** you need (other SQS/Lambda/email) to the topic.

3. **Get the Snowflake IAM policy snippet** that grants Snowflake permission to subscribe a Snowflake-managed SQS queue to your topic:

```sql
-- run this in Snowflake (replace with your SNS ARN)
SELECT SYSTEM$GET_AWS_SNS_IAM_POLICY('arn:aws:sns:us-east-1:111122223333:mytopic');
```

The function returns a JSON policy piece you must add to the SNS topic policy. (Snowflake docs explain this function.) ([Snowflake Documentation][6])

4. **Edit the SNS topic Access policy (AWS)** — open the topic → Edit → *Access policy* → merge the JSON returned by Snowflake into the topic policy and save. This step allows the Snowflake SQS queue (created by Snowflake when you create the external table) to subscribe to the SNS topic. ([Snowflake Documentation][7])

5. **Create your external table with the SNS ARN** — this tells Snowflake to subscribe its SQS queue to your SNS topic:

```sql
CREATE OR REPLACE EXTERNAL TABLE ext_table
  WITH LOCATION = @mystage/path1/
  FILE_FORMAT = (TYPE = JSON)
  AWS_SNS_TOPIC = 'arn:aws:sns:us-east-1:111122223333:mytopic';
```

When you run this, Snowflake will wire its SQS queue to the SNS topic (subscription) and handle receiving messages. ([Snowflake Documentation][5])

6. **Manually refresh once**:

```sql
ALTER EXTERNAL TABLE ext_table REFRESH;
```

After this, S3→SNS→(Snowflake SQS) will trigger automatic metadata refreshes for this external table. ([Snowflake Documentation][1])

**Alternative SNS path if you need Snowflake → SNS outbound notifications (advanced)**
Snowflake also supports **CREATE NOTIFICATION INTEGRATION** for outbound notifications to an SNS topic (this is a different feature used to let Snowflake publish events to SNS). That requires you to create an IAM role, run `CREATE NOTIFICATION INTEGRATION ... NOTIFICATION_PROVIDER = AWS_SNS`, then run `DESC NOTIFICATION INTEGRATION` to get `SF_AWS_IAM_USER_ARN` and `SF_AWS_EXTERNAL_ID` and update the IAM role trust policy accordingly. (This is used when Snowflake must assume an IAM role to send messages *outbound* to SNS.) See Snowflake docs for the full flow. ([Snowflake Documentation][2])

---

# Verification & debug checklist (copy/paste)

1. `SHOW EXTERNAL TABLES;` → check `notification_channel` (SQS ARN) and `last_refreshed_on`. ([Snowflake Documentation][3])
2. Upload a small file to the target S3 prefix → check:

   * S3 event log (or CloudTrail) to confirm event. ([AWS Documentation][4])
   * SQS messages (AWS Console → SQS) or SNS Delivery logs.
   * `SHOW EXTERNAL TABLES;` and `SELECT * FROM TABLE(information_schema.external_table_file_registration_history(...));` — check register/unregister events. ([Snowflake Documentation][1])
3. If nothing happens:

   * Ensure SNS topic policy includes Snowflake snippet from `SYSTEM$GET_AWS_SNS_IAM_POLICY`. ([Snowflake Documentation][6])
   * Ensure event notification in S3 is configured for correct prefix/pattern and target (SQS or SNS). ([AWS Documentation][4])
   * Ensure `AUTO_REFRESH` was not disabled (`AUTO_REFRESH = TRUE` is default). If ownership changed, `AUTO_REFRESH` can flip to `FALSE` — run `ALTER EXTERNAL TABLE ... SET AUTO_REFRESH = TRUE` if needed. ([Snowflake Documentation][8])

---

# Security & operational notes (must know)

* **SNS topic region** — create the SNS topic in the same region as your S3 bucket and Snowflake (reduce latency/egress). ([Snowflake Documentation][2])
* **IAM / topic policy edits** — merging Snowflake’s policy snippet into the SNS topic’s access policy is required for SNS approach. Use `SYSTEM$GET_AWS_SNS_IAM_POLICY` to generate the snippet. ([Snowflake Documentation][6])
* **No overlapping S3 notifications** — AWS does not allow overlapping queue configurations for the same prefix; plan your notifications carefully. ([Snowflake Documentation][1])
* **User-specified partitions** (manual partitions) disable auto-refresh. If you use manual partitions, you must `ALTER EXTERNAL TABLE ... ADD/REMOVE PARTITION` yourself. ([Snowflake Documentation][5])

---

# Short copy-paste cheat sheet

SQS path (quick):

```sql
-- create stage + external table (AUTO_REFRESH TRUE by default)
CREATE STAGE mystage URL='s3://mybucket/path' STORAGE_INTEGRATION = my_storage_int;
CREATE EXTERNAL TABLE ext_table WITH LOCATION=@mystage/path FILE_FORMAT=(TYPE=PARQUET);

-- show where Snowflake expects notifications (copy notification_channel)
SHOW EXTERNAL TABLES;

-- seed metadata
ALTER EXTERNAL TABLE ext_table REFRESH;
```

Then: in S3 console → Properties → Event notifications → create event → Destination: SQS → paste `notification_channel` ARN. ([Snowflake Documentation][1], [AWS Documentation][4])

SNS path (quick):

```sql
-- 1) Get IAM policy snippet to add to SNS policy:
SELECT SYSTEM$GET_AWS_SNS_IAM_POLICY('arn:aws:sns:...:mytopic');

-- 2) After adding the snippet in AWS SNS topic policy, create external table with SNS:
CREATE EXTERNAL TABLE ext_table
 WITH LOCATION=@mystage/path
 FILE_FORMAT=(TYPE=JSON)
 AWS_SNS_TOPIC='arn:aws:sns:...:mytopic';

-- 3) Manually seed metadata once:
ALTER EXTERNAL TABLE ext_table REFRESH;
```

Then ensure S3 event notifications publish to the SNS topic (S3 → Event notifications → send to SNS topic). ([Snowflake Documentation][5])

---

# Final checklist before you click “save” on the AWS console

* [ ] Are `ObjectCreated` (and `ObjectRemoved`, if needed) selected? (S3 event) ([AWS Documentation][4])
* [ ] Did you copy the **exact** SQS ARN from `SHOW EXTERNAL TABLES;` (no typos)? ([Snowflake Documentation][3])
* [ ] If using SNS: did you run `SYSTEM$GET_AWS_SNS_IAM_POLICY` and merge the returned JSON into topic access policy? ([Snowflake Documentation][6])
* [ ] Did you run `ALTER EXTERNAL TABLE ... REFRESH;` once (seed metadata)? ([Snowflake Documentation][1])

---

If you want, I can now:

* produce the **exact JSON snippets** you should paste into the SNS topic policy (I’ll generate the template with placeholders), and a ready-to-paste AWS Console checklist; **or**
* generate a **one-page runbook** that your ops team can use (with every click + SQL + expected outputs).

Which would you like me to generate now?

[1]: https://docs.snowflake.com/en/user-guide/tables-external-s3 "Refresh external tables automatically for Amazon S3 | Snowflake Documentation"
[2]: https://docs.snowflake.com/en/user-guide/notifications/creating-notification-integration-amazon-sns "Creating a notification integration to send notifications to an Amazon SNS topic | Snowflake Documentation"
[3]: https://docs.snowflake.com/en/sql-reference/sql/show-external-tables?utm_source=chatgpt.com "SHOW EXTERNAL TABLES - Snowflake Documentation"
[4]: https://docs.aws.amazon.com/AmazonS3/latest/userguide/enable-event-notifications.html?utm_source=chatgpt.com "Enabling and configuring event notifications using the Amazon S3 ..."
[5]: https://docs.snowflake.com/en/sql-reference/sql/create-external-table "CREATE EXTERNAL TABLE | Snowflake Documentation"
[6]: https://docs.snowflake.com/en/sql-reference/functions/system_get_aws_sns_iam_policy?utm_source=chatgpt.com "SYSTEM$GET_AWS_SNS_IAM_POLICY - Snowflake Documentation"
[7]: https://docs.snowflake.com/en/user-guide/data-load-snowpipe-auto-s3?utm_source=chatgpt.com "Automating Snowpipe for Amazon S3 - Snowflake Documentation"
[8]: https://docs.snowflake.com/en/user-guide/tables-external-auto "Refresh external tables automatically | Snowflake Documentation"
